# Domácí úkol:
Pomocí vlastních tříd a dědičnosti vytvořte "soubor tříd" = nástroj pro derivaci funkcí.

## Jak to má fungovat:

- derivace je jednoduše algoritmizovatelná, neboť se řídí prakticky pouze třemi pravidly (podílu se dá vyhnout pomocí násobení funkcí $\frac{1}{f(x)}$):
    - derivace součtu je součet derivací
    - pravidlo pro derivaci součinu
    - derivace funkce je derivovaná funkce násobená derivovaným argumentem

- tedy stačí implementovat tyto tři základní pravidla a pro každou elementární funkci (sin, cos, exp, ln, x^n, ...) vědět jak vypadá její derivace
- celý tento proces je možné reprezentovat pomocí tříd a dědičnosti
- trik je v tom, že se výrazy automaticky skládají do stromu (pro nás v podstatě skrytého), a tedy každý výraz je na venek pouze buď sčítání, násobení, nebo elementární funkce (a o jejich argumentech platí totéž)

## Podrobnější zadání:
Konkrétněji:
- Vytvořte třídu **Operator**
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor (zde stačí `pass`)
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
- Vytvořte třídu **BinaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `levy_argument` - levý operand
        - `pravy_argument` - pravý operand
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argumenty
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru
- Vytvořte třídu **UnaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `argument` - argument funkce 
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argument
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru

Pro počítání derivací použijte standardní pravidla:
- derivace součtu je součet derivací
- pravidlo pro derivaci součinu
- derivace funkce je derivovaná funkce násobená derivovaným argumentem

Pro **BinaryOperator** a **UnaryOperator** implementujte také metody implementující standardní operátory:
- `__add__()` - implementuje operátor `+`
- `__sub__()` - implementuje operátor `-`
- `__mul__()` - implementuje operátor `*`

Pro **BinaryOperator** implementujte dvě dceřiné třídy:
- **Addition** - implementuje operátor `+`
- **Multiplication** - implementuje operátor `*`

Pro **UnaryOperator** implementujte tyto dceřiné třídy:
- **exp** - implementuje funkci `exp`
- **sin** - implementuje funkci `sin`
- **cos** - implementuje funkci `cos`
- **ln** - implementuje funkci `ln`
- **frc** - implementuje funkci `1/x`
- **identity** - implementuje identitu `x`
- **cst** - implementuje konstantu `c`
- **mocnina** - implementuje mocninu `x^n`

U všech funckí implementujte adekvátně metodu `diff()` a `__str__()`, tak aby se funkce vypisovaly v přehledné podobě.

Ozkoušejte na následujících funkcích:
- $sin(x + cos(x*x)) + 1$
- $x^2 + 2x + 1 + 2x^2$
- $x^2 + 2x + exp(x)$

In [3]:
import math

class Operator:
    def __init__(self):
        self.strRep = ""
        self.diffRep = ""
    def __str__(self):
        return type(self).__name__
    
    def __repr__(self):
        return self.__str__()

    def __add__(self, other):
        return Addition(self, other)
    
    def __sub__(self, other):
        return Addition(self, Multiplication(cst(-1), other))
    
    def __mul__(self, other):
        return Multiplication(self, other)

    # def simplify(self):
    #     pass

class BinaryOperator(Operator):
    def __init__(self, leftArg, rightArg):
        self.leftArg = leftArg
        self.rightArg = rightArg


class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument
    
    def __str__(self):
        return type(self).__name__ + "(" + str(self.argument) + ")"


class Addition(BinaryOperator):
    def __str__(self):
        self.strRep = str(self.leftArg)  + " + " + str(self.rightArg)
        self.simplify()
        return self.strRep
    def diff(self):
        self.diffRep =  str((self.leftArg).diff()) + "+" + str( (self.rightArg).diff())
        self.simplify()
        return self.diffRep
    def simplify(self):
        #simmplify __str__()
        if( (str(self.leftArg) == "0") and (str(self.rightArg) == "0") ):
            self.strRep = str(0)
        if(str(self.leftArg) == "0"):
            self.strRep =  str(self.rightArg)
        if(str(self.rightArg) == "0"):
            self.strRep =  str(self.leftArg)
        #simplify diff()
        if( (str((self.leftArg).diff()) == "0") and (str((self.leftArg).diff()) == "0") ):
            self.diffRep = str(0)
        elif(str((self.leftArg).diff()) == "0"):
            self.diffRep =  str((self.rightArg).diff())
        elif(str((self.rightArg).diff()) == "0"):
            self.diffRep =  str((self.leftArg).diff())

class Multiplication(BinaryOperator):
    def __str__(self):
        self.strRep = "(" + str(self.leftArg) + ")" + "*" + "(" + str(self.rightArg) + ")"
        self.simplify()
        return self.strRep
    def diff(self):
        self.diffRep =  "(" + str((self.leftArg).diff()) + ")*("  + str(self.rightArg) +  ")+(" + str(self.leftArg) + ")*("  + str( (self.rightArg).diff()) + ")"
        self.simplify()
        return self.diffRep
    def simplify(self):
        #simmplify __str__()
        if( (str(self.leftArg) == "0") or (str(self.rightArg) == "0") ):
            self.strRep = str(0)
        elif( (str(self.leftArg) == "1") and (str(self.rightArg) == "1") ):
            self.strRep = str(1)
        elif(str(self.leftArg) == "1"):
            self.strRep = "(" + str(self.rightArg) + ")"
        elif(str(self.rightArg) == "1"):
            self.strRep = "(" + str(self.leftArg) + ")"
        #simplify diff(), x=0 or y=0 |-> 0
        if( (str(self.leftArg) == "0") or (str(self.rightArg) == "0") ):
            self.diffRep = str(0)
        #simplify diff(), x=1 and y=1 |-> 0
        elif( (str(self.leftArg) == "1") and (str(self.rightArg) == "1") ):
            self.diffRep = str(0) 
        #simplify diff(), x=1 |-> dy
        elif(str(self.leftArg) == "1"):
            self.diffRep = str((self.rightArg).diff())
        #simplify diff(), y=1 |-> dx
        elif(str(self.rightArg) == "1"):
            self.diffRep = str((self.leftArg).diff())
        #simplify diff(), dx=0 and dy=0 |-> 0
        elif( (str((self.leftArg).diff()) == "0") and (str((self.rightArg).diff()) == "0") ):
            self.diffRep = str(0)
        #simplify diff(), dx=0 and dy = 1 |-> x
        elif( (str((self.leftArg).diff()) == "0") and (str((self.rightArg).diff()) == "1") ):
            self.diffRep = "(" + str(self.leftArg) + ")"
        #simplify diff(), dx=0 |-> x*dy
        elif(str((self.leftArg).diff()) == "0"):
            self.diffRep = "(" + str(self.leftArg) + ")*("  + str( (self.rightArg).diff()) + ")"
        #simplify diff(), dy=0 and dx = 1 |-> y
        elif( (str((self.leftArg).diff()) == "1") and  (str((self.rightArg).diff()) == "0")):
            self.diffRep = "(" + str(self.rightArg) +  ")"
        #simplify diff(), dy=0 |-> dx*y
        elif(str((self.rightArg).diff()) == "0"):
            self.diffRep = "(" + str((self.leftArg).diff()) + ")*("  + str(self.rightArg) +  ")"
        #simplify diff(), dx=1 and dy = 1 |-> x+y
        elif( (str((self.leftArg).diff()) == "1") and (str((self.rightArg).diff()) == "1")  ):
            self.diffRep = "(" + str(self.leftArg) + "+" + str(self.rightArg) + ")"
        #simplify diff(), dx=1 |-> y + x*dy
        elif( str((self.leftArg).diff()) == "1"):
            self.diffRep = "(" + str(self.rightArg) + "+" +  "("+str(self.leftArg)+")*("+str((self.rightArg).diff())+")" + ")"
        #simplify diff(), dy=1 |-> dx*y + x
        elif(str((self.rightArg).diff()) == "1"):
            self.diffRep = "("+ "("+str((self.leftArg).diff())+")*("+str(self.rightArg)+")" + "+" + str(self.leftArg) + ")"


class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return str(self.value)
    def diff(self):
        return str(0)        

class identity(UnaryOperator):
    def __init__(self, ):
        pass        
    def __str__(self):
        #this is probably a problematic part, gets name of the variable (instance of the class Identity), from globals namespace, 
        #will not work if class is instantiated inside function, but i don't know how to do the same for locals namespace
        #alternative solution will be calling Identity with the explicit argument - name of the variable, see second version of Identity below
        varName = [k for k,v in globals().items() if v is self]
        return str(varName[0])
    def diff(self):
        return str(1)
# class Identity(UnaryOperator):
#     #alternative solution to above mentioned problem, Identity is called with the name of variable as an argument
#     #this version works also inside the functions, but does not comply with the requirements of the assignment
#     def __init__(self, varName):
#         self.arg = varName 
#     def __str__(self):
#         return str(self.arg)
#     def diff(self):
#         return str(1)
      
class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent
    def __str__(self):
        self.strRep = "(" + str(self.argument) + ")^" + str(self.exponent)
        self.simplify()
        return self.strRep
    def diff(self):
        self.diffRep =  "("+"("+str(self.exponent)+")"+"*"+"("+"("+str(self.argument)+")"+"^"+str(self.exponent-1)+")"+")"+"*"+"("+str((self.argument).diff())+")"
        self.simplify()
        return self.diffRep   
    def simplify(self):
        #simplify __str__
        if(str(self.argument) == "0"):
            self.strRep = str(0)
        elif( (str(self.argument) == "1") or (str(self.exponent) == "0") ):
            self.strRep = str(1)
        elif(str(self.exponent) == "1"):
            self.strRep = "(" + str(self.argument) + ")"
        #simplify diff(): arg=0 or exp=0 or darg=0
        if( (str(self.argument) == "0") or (str(self.argument) == "1") or (str(self.exponent) == "0") or (str((self.argument).diff()) == "0") ):
            self.diffRep = str(0)
        #simplify diff(): exp=1
        elif(str(self.exponent) == "1"):
            self.diffRep = "("+str((self.argument).diff())+")"
        #simplify diff(): exp=2 and darg = 1
        elif( (str(self.exponent) == "2") and (str((self.argument).diff()) == "1") ):
            self.diffRep = "("+ "("+str(self.exponent)+")"+"*"+"("+str(self.argument)+")" +")"
        #simplify diff(): exp=2
        elif(str(self.exponent) == "2"):
            self.diffRep =  "("+str(self.exponent)+")" +"*"+ "("+str(self.argument)+")" +"*"+  "("+str((self.argument).diff())+")"
        #simplify diff(): darg = 
        elif(str((self.argument).diff()) == "1"):
            self.diffRep = "("+"("+str(self.exponent)+")"+"*"+"("+"("+str(self.argument)+")"+"^"+str(self.exponent-1)+")"+")"

class sin(UnaryOperator):
    def diff(self):
        self.diffRep =  "(" + str(cos.__name__) + "(" + str(self.argument) + ")" + ")" + "*" + "(" + str((self.argument).diff()) + ")"
        self.simplify()
        return self.diffRep
    def simplify(self):
        #simplify diff(): darg=0) |-> 0
        if(str((self.argument).diff()) == "0"):
            self.diffRep = str(0)
        #simplify diff(): darg=1) |-> cos(arg)
        elif(str((self.argument).diff()) == "1"):
            self.diffRep =  "(" + str(cos.__name__) + "(" + str(self.argument) + ")" + ")"

class cos(UnaryOperator):
    def diff(self):
        self.diffRep =  "(" + "(-1)*" + "(" + str(sin.__name__) + "(" + str(self.argument) + ")" + ")" + "*" + "(" + str((self.argument).diff()) + ")"
        self.simplify()
        return self.diffRep
    def simplify(self):
        #simplify diff(): darg=0) |-> 0
        if(str((self.argument).diff()) == "0"):
            self.diffRep = str(0)
        #simplify diff(): darg=1) |-> (-1)*sin(arg)
        elif(str((self.argument).diff()) == "1"):
            self.diffRep =  "(" + "(-1)*" + "(" + str(sin.__name__) + "(" + str(self.argument) + ")" + ")"

class exp(UnaryOperator):
    def diff(self):
        self.diffRep =  "(" + str((self.argument).diff())  + "*" +  type(self).__name__ + "(" + str(self.argument)  + ")" + ")"
        self.simplify()
        return self.diffRep
    def simplify(self):
        #simplify diff(): darg = 0 |-> 0
        if(str((self.argument).diff()) == "0"):
            self.diffRep = str(0)
        #simplify diff(): darg = 1 |-> exp(arg)
        elif(str((self.argument).diff()) == "1"):
            self.diffRep =  "(" +  type(self).__name__ + "(" + str(self.argument)  + ")" + ")"

class ln(UnaryOperator):
    def diff(self):
        self.diffRep =  "(" + str((self.argument).diff())  + "*" +  "(1/"+ "("+str(self.argument)+")" +")" + ")" 
        self.simplify()
        return self.diffRep             
    def simplify(self):
        #simplify diff(): darg = 0 |-> 0
        if(str((self.argument).diff()) == "0"):
            self.diffRep = str(0)
        #simplify diff(): darg = 1 |-> 1/ln(x)
        elif(str((self.argument).diff()) == "1"):
            self.diffRep = "(1/"+ "("+str(self.argument)+")" +")" 
        
class frc(UnaryOperator):
    def __str__(self):
        self.strRep = "1/" + "(" + str(self.argument) + ")"
        self.simplify()
        return self.strRep
    def diff(self):
        self.diffRep = "("+"-1"+")" + "*" + "(" + "1/" + "("+str(mocnina(self.argument, 2))+")" + ")" + "*" + "(" + str((self.argument).diff()) + ")"
        self.simplify()
        return self.diffRep 
    def simplify(self):
        #simplify __str__: arg = 1 |-> 1
        if(str(self.argument) == "1"):
            self.strRep = str(1)
        #simplify diff(): darg = 0 |-> 0
        if(str(str((self.argument).diff())) == "0"): 
            self.diffRep = str(0)
        #simplify diff(): darg = 1 |-> (-1)*1/(arg^2)
        elif(str(str((self.argument).diff())) == "1"):
            self.diffRep = "("+"-1"+")" + "*" + "(" + "1/" + "("+str(mocnina(self.argument, 2))+")" + ")"

In [4]:
# takto by se to mělo používat
x = identity()
# sin(x + cos(x*x)) + 1
f1 = sin(x + cos(x*x)) + cst(1)
print(f1)
print(f1.diff())

sin(x + cos((x)*(x))) + 1
(cos(x + cos((x)*(x))))*(1+((-1)*(sin((x)*(x)))*((x+x)))


můj výstup byl:

`sin(x + cos((x)*(x))) + 1`

`(cos(x + cos((x)*(x))))*(1 + ((-1)*(sin((x)*(x))))*((1)*(x) + (x)*(1))) + 0`

In [5]:
x = identity()
# x ^ 2 + 2x + 1 + 2x ^ 2
f2 = mocnina(x,2) + cst(2)*x + cst(1) + cst(2)*mocnina(x,2)
print(f2)
print(f2.diff())


(x)^2 + (2)*(x) + 1 + (2)*((x)^2)
((2)*(x))+(2)+(2)*(((2)*(x)))


můj výstup byl:

`(x)^2 + (2)*(x) + 1 + (2)*((x)^2)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + 0 + (0)*((x)^2) + (2)*(((2)*((x)^1))*(1))`

In [6]:
x = identity()
# x ^ 2 + 2x + exp(x)
f3 = mocnina(x,2) + cst(2)*x + exp(x)
print(f3)
print(f3.diff())


(x)^2 + (2)*(x) + exp(x)
((2)*(x))+(2)+(exp(x))


můj výstup byl:

`(x)^2 + (2)*(x) + exp(x)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + (exp(x))*(1)`


# Bonusový úkol:
Doplňte ke všem třídám metodu `simplify()`, která provede zjednodušení výrazu. Například:
- součet s nulou vrátí druhý operand
- součin s nulou vrátí nulu
- mocnina s exponentem 0 vrátí 1
- mocnina s exponentem 1 vrátí první argument
- součín dvou stejných argumentů vrátí mocninu s exponentem 2
- součin konstant vrátí konstantu se součinem
- ...

Doplňte metody `__add__, __radd__, __mul__, __rmul__, ...` tak aby podporovaly operace s čísly (např. `2 + x` nebo `x * 3`), a zjednodušovaly zadávání výrazů. 